# TensorFlow

TensorFlow, as the name indicates, is a framework to define and run computations involving tensors. A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

In [ ]:
# installation command
#!pip install tensorflow
#!pip install numpy

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
a=3 #[treated as a 0-D tensor]
a=[3,5] #[treated as a 1-D tensor]
a=[[3,5],[1,6]] # treated as a 2-D sensor

A tf.Tensor has the following properties:
    1. a data type (float32, int32, or string, for example)
    2. a shape
Some types of tensors are special, the main ones are:
    1. tf.Variable
    2. tf.constant
    3. tf.placeholder

In [ ]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Notice that printing the tensors does not output the values 3.0, 4.0, and 7.0 as you might expect. The above statements only build the computation graph. These tf.Tensor objects just represent the results of the operations that will be run.

   To evaluate tensors, instantiate a tf.Session object, informally known as a session. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations.

In [ ]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
sum_1 = a + b
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(sum_1))

In [ ]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) 
total = a + b
writer = tf.summary.FileWriter('vp1')
writer.add_graph(tf.get_default_graph())


In [ ]:
# variable......
a=tf.Variable(6)
b=tf.Variable(5)
f=a+b
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    print(sess.run(f))

In [ ]:
# constant....
import tensorflow as tf
a=tf.constant([1,2,3])
b=tf.constant([5,6,7])
z=tf.multiply(a,b)
with tf.Session() as sess:
    print(sess.run(z))

In [ ]:
## place holders............
import tensorflow as tf
a=tf.placeholder(tf.float32)
b=tf.placeholder(tf.float32)
z=a+b
with tf.Session() as sess:
    print(sess.run(z,feed_dict={a:[1,2,3],b:[1,2,3]}))

## Linear Regression

In [ ]:
# Model Parameters
w=tf.Variable([.3],tf.float32)
b=tf.Variable([-.3],tf.float32)


In [ ]:
# input and output 
x=tf.placeholder(tf.float32)
y=tf.placeholder(tf.float32)

In [ ]:
# Model
linear_model=w*x+b

# loss
squared_delta=tf.square(linear_model-y)
loss=tf.reduce_sum(squared_delta)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))


In [ ]:
## Linear regression with optimization

w=tf.Variable([.3],tf.float32)
b=tf.Variable([-.3],tf.float32)
x=tf.placeholder(tf.float32)
linear_model=w*x+b
y=tf.placeholder(tf.float32)
squared_delta=tf.square(linear_model-y)
loss=tf.reduce_sum(squared_delta)

In [ ]:
## optimizer
optimizer=tf.train.GradientDescentOptimizer(0.01)
train=optimizer.minimize(loss)


In [ ]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(0,1000):
        sess.run(train,{x:[1,2,3,4],y:[0,-1,-2,-3]})
        if i%100==0:
            print('loss:', sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))
    print('final value of W: ', sess.run(w))
    print('final value of b: ', sess.run(b))
    print('final loss:', sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

## DNN with MNIST Data

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. 

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data",one_hot=True) # loading data from TenserFlow's data directory

In [ ]:
# model parameters
n_nodes_hl1=500
n_nodes_hl2=500
n_nodes_hl3=500
n_classes=10
batch=100

In [ ]:
# model's input/ output
x=tf.placeholder('float',[None,784])
y=tf.placeholder('float')

In [ ]:
# model architecture

def neuralnet(data):
    hidden_layer1={'weights':tf.Variable(tf.random_normal([784,n_nodes_hl1])),
                   'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_layer2={'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
                  'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_layer3={'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                  'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer={'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                 'biases':tf.Variable(tf.random_normal([n_classes]))}
    l1=tf.add(tf.matmul(data,hidden_layer1['weights']),hidden_layer1['biases'])
    l1=tf.nn.relu(l1)
    l2=tf.add(tf.matmul(l1,hidden_layer2['weights']),hidden_layer2['biases'])
    l2=tf.nn.relu(l2)
    l3=tf.add(tf.matmul(l2,hidden_layer3['weights']),hidden_layer3['biases'])
    l3=tf.nn.relu(l3)
    out=tf.add(tf.matmul(l3,output_layer['weights']),output_layer['biases'])
    return(out)


In [ ]:
# model training

def train_NN(x):
    predict=neuralnet(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict,labels=y))
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    num_epochs=10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(num_epochs):
            epoch_loss=0
            for i in range(int(mnist.train.num_examples/batch)):
                batch_x,batch_y=mnist.train.next_batch(batch)
                _,batch_cost=sess.run([optimizer,cost], feed_dict={x:batch_x,y:batch_y})
                epoch_loss+=batch_cost
            print('epoch', epoch, 'completed out of', num_epochs, 'loss', epoch_loss )
        correct=tf.equal(tf.arg_max(predict,1),tf.arg_max(y,1))
        accuracy=tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))

        
train_NN(x)

## CNN with MNIST Data

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data",one_hot=True)

In [ ]:
batch_size=128
x=tf.placeholder(float,[None,784])
y=tf.placeholder(float)
n_classes=10

In [ ]:
def conv2D(x,w):
    return(tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME'))
def maxpool2D(x):
    return(tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME'))

In [ ]:
def cnn_net(x):
    weights={'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
             'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
             'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
             'W_out':tf.Variable(tf.random_normal([1024,n_classes]))
              }
    
    biases={'b_conv1':tf.Variable(tf.random_normal([32])),
            'b_conv2':tf.Variable(tf.random_normal([64])),
            'b_fc':tf.Variable(tf.random_normal([1024])),
            'b_out':tf.Variable(tf.random_normal([n_classes]))
        }
    x=tf.reshape(x,shape=[-1,28,28,1])
    layer1=conv2D(x,weights['W_conv1'])+biases['b_conv1']
    layer1=maxpool2D(layer1)
    layer1=tf.nn.relu(layer1)
    
    layer2=conv2D(layer1,weights['W_conv2'])+biases['b_conv2']
    layer2=maxpool2D(layer2)
    layer2=tf.nn.relu(layer2)
    
    
    layer_fc=tf.reshape(layer2,[-1,7*7*64])
    layer_fc=tf.add(tf.matmul(layer_fc,weights['W_fc']),biases['b_fc'])
    layer_fc=tf.nn.relu(layer_fc)
    
    layer_out=tf.add(tf.matmul(layer_fc,weights['W_out']),biases['b_out'])
    return(layer_out)
    
    
    
    

In [ ]:
def train_cnn_net(x):
    predict=cnn_net(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict,labels=y))
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    num_epochs=10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(num_epochs):
            epoch_loss=0
            for i in range(int(mnist.train.num_examples/batch_size)):
                apoch_x,apoch_y=mnist.train.next_batch(batch_size)
                k,c=sess.run([optimizer,cost], feed_dict={x:apoch_x,y:apoch_y})
                epoch_loss+=c
            print('epoch', epoch, 'completed out of', num_epochs, 'loss', epoch_loss )
        correct=tf.equal(tf.arg_max(predict,1),tf.arg_max(y,1))
        accuracy=tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))

        
train_cnn_net(x)

## Autoencoder

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import fully_connected

mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

In [ ]:
num_inputs=784    #28x28 pixels
num_hid1=392
num_hid2=196
num_hid3=num_hid1
num_output=num_inputs
lr=0.01
actf=tf.nn.relu

In [ ]:
X=tf.placeholder(tf.float32,shape=[None,num_inputs])
initializer=tf.variance_scaling_initializer()

w1=tf.Variable(initializer([num_inputs,num_hid1]),dtype=tf.float32)
w2=tf.Variable(initializer([num_hid1,num_hid2]),dtype=tf.float32)
w3=tf.Variable(initializer([num_hid2,num_hid3]),dtype=tf.float32)
w4=tf.Variable(initializer([num_hid3,num_output]),dtype=tf.float32)

b1=tf.Variable(tf.zeros(num_hid1))
b2=tf.Variable(tf.zeros(num_hid2))
b3=tf.Variable(tf.zeros(num_hid3))
b4=tf.Variable(tf.zeros(num_output))

hid_layer1=actf(tf.matmul(X,w1)+b1)
hid_layer2=actf(tf.matmul(hid_layer1,w2)+b2)
hid_layer3=actf(tf.matmul(hid_layer2,w3)+b3)
output_layer=actf(tf.matmul(hid_layer3,w4)+b4)

In [ ]:
loss=tf.reduce_mean(tf.square(output_layer-X))

optimizer=tf.train.AdamOptimizer(lr)
train=optimizer.minimize(loss)

init=tf.global_variables_initializer()

In [ ]:
num_epoch=5
batch_size=150
num_test_images=10

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epoch):
        
        num_batches=mnist.train.num_examples//batch_size
        for iteration in range(num_batches):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            sess.run(train,feed_dict={X:X_batch})
            
        train_loss=loss.eval(feed_dict={X:X_batch})
    results=output_layer.eval(feed_dict={X:mnist.test.images[:num_test_images]})

print("epoch {} loss {}".format(epoch,train_loss))

In [ ]:
#Comparing original images with reconstructions
f,a=plt.subplots(2,10,figsize=(20,4))
for i in range(num_test_images):
    a[0][i].imshow(np.reshape(mnist.test.images[i],(28,28)))
    a[1][i].imshow(np.reshape(results[i],(28,28)))